In [1]:
from itertools import combinations
from tqdm.auto import tqdm
import numpy as np
import sample as sampler
import utils
import attacker

## Generate puzzle with given error_rate

generate n isometrices(sketches) with noise. Assume the original template is w, then the output for each sketching is Matrix = SS(w + noise) and the correponding codeword as b satisfying Matrix * (w + noise) = b.

In [69]:
# codeword parameters
dimension = 512
k = 2
n = 511
error_rate = 0.1    # angle(w + noise_1, w + noise_2) approximate arccos(error_rate * \sqrt(2))

In [70]:
w, bs, isometric_matrixes, _ = sampler.generate_puzzle_n(dimension, k, error_rate=error_rate, n = n)

Generating Puzzles:   0%|          | 0/511 [00:00<?, ?it/s]

In [30]:
# get the mean angle between each pair of templates, i.e. angle(w + noise_1, w + noise_2)

angles = []
ws = [M.T @ b for M, b in zip(isometric_matrixes, bs)]
for i, j in tqdm(combinations(range(len(ws)), 2)):
    angles.append(utils.get_angle_of_two_vectors(ws[i], ws[j]))

print("Mean Degree: {:.2f}{}".format(np.mean(angles), u'\u00b0'))

0it [00:00, ?it/s]

Mean Degree: 8.06°


## solving original template knowing zero positions of each codeword

svd solver

In [58]:
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, bs, dimension, k, threshold=20, max_rtimes=1000,
                                                                    algorithm="SVD", disable_tqdm=False, k_each_matrix=2)

Solving(SVD):   0%|          | 0/1000 [00:00<?, ?it/s]

In [27]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, bs[0]) or np.allclose(b, -bs[0]))

The result is:  True


lsa solver

In [66]:
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, bs, dimension, k, threshold=20, max_rtimes=10000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, error_rate = error_rate * 3.0)

Solving(LSA):   0%|          | 0/10000 [00:00<?, ?it/s]

In [68]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, bs[0]) or np.allclose(b, -bs[0]))

The result is:  True


## solving original template without knowing zero positions of each codeword

In [98]:
# toy example for small k = 2
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, k, threshold=20, max_rtimes=1000,
                                                        algorithm="SVD", disable_tqdm=False, k_each_matrix=1, scale=3.0)

Solving(SVD):   0%|          | 0/22 [00:00<?, ?it/s]

In [100]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, bs[0]) or np.allclose(b, -bs[0]))

The result is:  True


In [105]:
# toy example for small k = 2
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, k, threshold=20, max_rtimes=1000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, scale=1000.0, error_rate=error_rate * 3.0)

Solving(LSA):   0%|          | 0/7389 [00:00<?, ?it/s]

In [106]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, bs[0]) or np.allclose(b, -bs[0]))

The result is:  True
